In [4]:
import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

In [5]:
# Figur erstellen
fig = go.FigureWidget()

# Überschriften
vector1_title = widgets.HTML(value='<h3>Vektor 1</h3>')
vector2_title = widgets.HTML(value='<h3>Vektor 2</h3>')

# Slider für Vektor 1
x_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Y:')
z_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Z:')

# Slider für Vektor 2
x2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='X2:')
y2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Y2:')
z2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Z2:')

# Checkboxen
show_grid = widgets.Checkbox(value=False, description='Gitter anzeigen')
show_grid_numbers = widgets.Checkbox(value=False, description='Gitter-Beschriftung')
show_axis_numbers = widgets.Checkbox(value=True, description='Achsen-Beschriftung')
show_second_vector = widgets.Checkbox(value=False, description='Zweiten Vektor anzeigen')

# Neue Funktions-Buttons
calc_cross_product = widgets.Button(description='Kreuzprodukt')
calc_angle = widgets.Button(description='Winkel')
show_projection = widgets.Button(description='Projektion')
rotation_slider = widgets.FloatSlider(value=0, min=0, max=360, description='Rotation (°):')
scale_slider = widgets.FloatSlider(value=1, min=0.1, max=5, description='Skalierung:')

# Info-Widget
vector_info = widgets.HTML(
    value='<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">'
          '<h3>Vektor Information:</h3>'
          '<p>Initialisierung...</p></div>'
)

In [6]:
def update_3d_plot(*args):
    # Vektoren berechnen
    v1 = [x_slider.value, y_slider.value, z_slider.value]
    v2 = [x2_slider.value, y2_slider.value, z2_slider.value]
    v_sum = [v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2]]
    
    with fig.batch_update():
        fig.data = []  # Reset data
        
        # Koordinatenachsen
        axis_length = 5
        fig.add_scatter3d(x=[-axis_length, axis_length], y=[0, 0], z=[0, 0],
                         mode='lines', line=dict(color='red', width=2),
                         name='X-Achse')
        fig.add_scatter3d(x=[0, 0], y=[-axis_length, axis_length], z=[0, 0],
                         mode='lines', line=dict(color='green', width=2),
                         name='Y-Achse')
        fig.add_scatter3d(x=[0, 0], y=[0, 0], z=[-axis_length, axis_length],
                         mode='lines', line=dict(color='blue', width=2),
                         name='Z-Achse')
        
        # Erster Vektor
        fig.add_cone(x=[v1[0]], y=[v1[1]], z=[v1[2]], 
                    u=[v1[0]], v=[v1[1]], w=[v1[2]],
                    sizeref=0.2, showscale=False, 
                    colorscale=[[0, 'blue'], [1, 'blue']])
        fig.add_scatter3d(x=[0, v1[0]], y=[0, v1[1]], z=[0, v1[2]],
                         mode='lines', line=dict(color='blue', width=4),
                         name=f'Vektor 1 ({v1[0]:.2f}, {v1[1]:.2f}, {v1[2]:.2f})')
        
        # Zweiter Vektor wenn aktiviert
        if show_second_vector.value:
            fig.add_cone(x=[v2[0]], y=[v2[1]], z=[v2[2]], 
                        u=[v2[0]], v=[v2[1]], w=[v2[2]],
                        sizeref=0.2, showscale=False, 
                        colorscale=[[0, 'red'], [1, 'red']])
            fig.add_scatter3d(x=[0, v2[0]], y=[0, v2[1]], z=[0, v2[2]],
                            mode='lines', line=dict(color='red', width=4),
                            name=f'Vektor 2 ({v2[0]:.2f}, {v2[1]:.2f}, {v2[2]:.2f})')
            
            # Summenvektor
            fig.add_scatter3d(x=[0, v_sum[0]], y=[0, v_sum[1]], z=[0, v_sum[2]],
                            mode='lines', line=dict(color='green', width=4, dash='dot'),
                            name=f'Summe ({v_sum[0]:.2f}, {v_sum[1]:.2f}, {v_sum[2]:.2f})')
        
        # Layout aktualisieren
        camera = fig.layout.scene.camera
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-5, 5], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                yaxis=dict(range=[-5, 5], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                zaxis=dict(range=[-5, 5], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                aspectmode='cube',
                camera=camera
            ),
            width=800,
            height=800
        )
        
        # Info aktualisieren
        info_html = f'''
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
            <h3>Vektor Information:</h3>
            <p>Vektor 1: ({v1[0]:.2f}, {v1[1]:.2f}, {v1[2]:.2f})</p>
            <p>Berechnung der Länge:</p>
            <p>√({v1[0]:.2f}² + {v1[1]:.2f}² + {v1[2]:.2f}²) = {np.sqrt(v1[0]**2 + v1[1]**2 + v1[2]**2):.2f}</p>
            {f"<p>Vektor 2: ({v2[0]:.2f}, {v2[1]:.2f}, {v2[2]:.2f})</p>" if show_second_vector.value else ""}
            {f"<p>Summenvektor: ({v_sum[0]:.2f}, {v_sum[1]:.2f}, {v_sum[2]:.2f})</p>" if show_second_vector.value else ""}
        </div>
        '''
        vector_info.value = info_html

In [7]:
# Slider-Gruppen
vector1_sliders = widgets.VBox([vector1_title, x_slider, y_slider, z_slider])
vector2_sliders = widgets.VBox([vector2_title, x2_slider, y2_slider, z2_slider])
vector2_sliders.layout.visibility = 'hidden'

# Checkboxen
checkbox_container = widgets.VBox([
    widgets.HBox([show_grid]),
    widgets.HBox([show_grid_numbers]),
    widgets.HBox([show_axis_numbers]),
    widgets.HBox([show_second_vector])
])

# Funktions-Buttons
function_container = widgets.VBox([
    widgets.HBox([calc_cross_product, calc_angle, show_projection]),
    widgets.HBox([rotation_slider, scale_slider])
])

# Event Handler für zweiten Vektor
def on_show_second_vector_change(change):
    vector2_sliders.layout.visibility = 'visible' if change.new else 'hidden'

# Verbindungen
show_second_vector.observe(on_show_second_vector_change, names='value')
for w in [x_slider, y_slider, z_slider, x2_slider, y2_slider, z2_slider,
          show_grid, show_grid_numbers, show_axis_numbers, show_second_vector]:
    w.observe(update_3d_plot, names='value')

# Hauptcontainer
container = widgets.VBox([
    widgets.HBox([vector1_sliders, vector2_sliders]),
    checkbox_container,
    function_container,
    fig,
    vector_info
])

# Anzeigen
display(container)
update_3d_plot()

In [8]:
# Hilfsfunktionen
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

def cross_product(v1, v2):
    return np.cross(v1, v2)

def angle_between(v1, v2):
    v1_n = normalize(v1)
    v2_n = normalize(v2)
    return np.arccos(np.clip(np.dot(v1_n, v2_n), -1.0, 1.0))

def rotate_vector(v, angle, axis='z'):
    theta = np.radians(angle)
    c, s = np.cos(theta), np.sin(theta)
    if axis == 'z':
        R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])
    elif axis == 'y':
        R = np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    else:  # x-axis
        R = np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    return np.dot(R, v)

# Event Handler für Funktionen
def on_cross_product_click(b):
    v1 = np.array([x_slider.value, y_slider.value, z_slider.value])
    v2 = np.array([x2_slider.value, y2_slider.value, z2_slider.value])
    cross = cross_product(v1, v2)
    
    with fig.batch_update():
        # Kreuzprodukt als lila Vektor darstellen
        fig.add_scatter3d(x=[0, cross[0]], y=[0, cross[1]], z=[0, cross[2]],
                         mode='lines', line=dict(color='purple', width=4),
                         name=f'Kreuzprodukt ({cross[0]:.2f}, {cross[1]:.2f}, {cross[2]:.2f})')

def on_angle_click(b):
    v1 = np.array([x_slider.value, y_slider.value, z_slider.value])
    v2 = np.array([x2_slider.value, y2_slider.value, z2_slider.value])
    angle = np.degrees(angle_between(v1, v2))
    
    # Winkelinformation zum vector_info hinzufügen
    current_info = vector_info.value
    angle_info = f'<p><b>Winkel zwischen Vektoren:</b> {angle:.2f}°</p></div>'
    vector_info.value = current_info.replace('</div>', angle_info)

def on_projection_click(b):
    v1 = np.array([x_slider.value, y_slider.value, z_slider.value])
    v2 = np.array([x2_slider.value, y2_slider.value, z2_slider.value])
    
    # Projektion von v1 auf v2
    v2_norm = normalize(v2)
    proj = np.dot(v1, v2_norm) * v2_norm
    
    with fig.batch_update():
        # Projektion als gestrichelten orangenen Vektor darstellen
        fig.add_scatter3d(x=[0, proj[0]], y=[0, proj[1]], z=[0, proj[2]],
                         mode='lines', line=dict(color='orange', width=4, dash='dash'),
                         name=f'Projektion ({proj[0]:.2f}, {proj[1]:.2f}, {proj[2]:.2f})')

# Button-Handler verbinden
calc_cross_product.on_click(on_cross_product_click)
calc_angle.on_click(on_angle_click)
show_projection.on_click(on_projection_click)

# Rotations-Handler
def on_rotation_change(change):
    v1 = np.array([x_slider.value, y_slider.value, z_slider.value])
    rotated = rotate_vector(v1, rotation_slider.value)
    x_slider.value, y_slider.value, z_slider.value = rotated

rotation_slider.observe(on_rotation_change, names='value')

In [9]:
# Erweiterte Funktions-Buttons
advanced_functions = widgets.VBox([
    widgets.HBox([calc_cross_product, calc_angle, show_projection]),
    widgets.HBox([
        widgets.VBox([
            widgets.HTML(value='<b>Transformationen:</b>'),
            rotation_slider,
            scale_slider
        ])
    ])
])

# Erweiterter Hauptcontainer
container = widgets.VBox([
    widgets.HBox([vector1_sliders, vector2_sliders]),
    checkbox_container,
    advanced_functions,
    fig,
    vector_info
])

# Anzeigen
display(container)
update_3d_plot()

In [10]:
import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Figur EINMAL erstellen
fig = go.FigureWidget()

# Slider für 3D Vektor
x_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Y:')
z_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Z:')

def update_3d_plot(*args):
    x = x_slider.value
    y = y_slider.value
    z = z_slider.value
    
    with fig.batch_update():
        if len(fig.data) == 0:
            # Vektor mit Pfeilspitze
            fig.add_cone(x=[x], y=[y], z=[z], u=[x], v=[y], w=[z],
                        sizeref=0.2, showscale=False, colorscale=[[0, 'blue'], [1, 'blue']])
            
            # Vektor-Linie
            fig.add_scatter3d(x=[0, x], y=[0, y], z=[0, z],
                            mode='lines',
                            line=dict(color='blue', width=4),
                            showlegend=False)
            
            # Koordinatenachsen
            axis_length = 5
            # X-Achse
            fig.add_scatter3d(x=[-axis_length, axis_length], y=[0, 0], z=[0, 0],
                            mode='lines',
                            line=dict(color='red', width=2),
                            name='X-Achse')
            # Y-Achse
            fig.add_scatter3d(x=[0, 0], y=[-axis_length, axis_length], z=[0, 0],
                            mode='lines',
                            line=dict(color='green', width=2),
                            name='Y-Achse')
            # Z-Achse
            fig.add_scatter3d(x=[0, 0], y=[0, 0], z=[-axis_length, axis_length],
                            mode='lines',
                            line=dict(color='blue', width=2),
                            name='Z-Achse')
            
            # Achsen-Beschriftungen
            for i in range(-5, 6):
                if i != 0:
                    # X-Achsen-Markierungen
                    fig.add_scatter3d(x=[i], y=[0], z=[0],
                                    mode='text',
                                    text=[str(i)],
                                    textposition='bottom center',
                                    showlegend=False)
                    # Y-Achsen-Markierungen
                    fig.add_scatter3d(x=[0], y=[i], z=[0],
                                    mode='text',
                                    text=[str(i)],
                                    textposition='bottom center',
                                    showlegend=False)
                    # Z-Achsen-Markierungen
                    fig.add_scatter3d(x=[0], y=[0], z=[i],
                                    mode='text',
                                    text=[str(i)],
                                    textposition='bottom center',
                                    showlegend=False)
        else:
            # Vektor aktualisieren
            fig.data[0].x = [x]
            fig.data[0].y = [y]
            fig.data[0].z = [z]
            fig.data[0].u = [x]
            fig.data[0].v = [y]
            fig.data[0].w = [z]
            fig.data[1].x = [0, x]
            fig.data[1].y = [0, y]
            fig.data[1].z = [0, z]
        
        # Layout aktualisieren
        camera = fig.layout.scene.camera
        fig.update_layout(
            scene=dict(
				xaxis=dict(range=[-5, 5], gridcolor='lightgray', showbackground=False),
        		yaxis=dict(range=[-5, 5], gridcolor='lightgray', showbackground=False),
        		zaxis=dict(range=[-5, 5], gridcolor='lightgray', showbackground=False),
                aspectmode='cube',
                camera=camera,
            ),
            showlegend=True,
            width=800,  # Größere Darstellung
            height=800
        )

# Slider verbinden
x_slider.observe(update_3d_plot, names='value')
y_slider.observe(update_3d_plot, names='value')
z_slider.observe(update_3d_plot, names='value')

# Layout erstellen
container = widgets.VBox([
    widgets.HBox([x_slider, y_slider, z_slider]),
    fig
])
display(container)

# Initiale Darstellung
update_3d_plot()

In [11]:
import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Figur erstellen
fig = go.FigureWidget()

# Überschriften
vector1_title = widgets.HTML(value='<h3>Vektor 1</h3>')
vector2_title = widgets.HTML(value='<h3>Vektor 2</h3>')

# Slider für Vektor 1
x_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Y:')
z_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Z:')

# Slider für Vektor 2
x2_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='X2:')
y2_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Y2:')
z2_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Z2:')

# Checkboxen
show_grid = widgets.Checkbox(value=False, description='Gitter anzeigen')
show_axis_numbers = widgets.Checkbox(value=True, description='Achsen-Beschriftung')
show_second_vector = widgets.Checkbox(value=False, description='Zweiten Vektor anzeigen')
show_sum_vector = widgets.Checkbox(value=False, description='Summenvektor anzeigen')

# Funktions-Buttons mit Erklärungen
calc_cross_product = widgets.Button(
    description='Kreuzprodukt',
    tooltip='Berechnet den Vektor, der senkrecht auf beiden Eingabevektoren steht'
)
calc_angle = widgets.Button(
    description='Winkel',
    tooltip='Berechnet den Winkel zwischen beiden Vektoren'
)
show_projection = widgets.Button(
    description='Projektion',
    tooltip='Zeigt die Projektion des ersten Vektors auf den zweiten Vektor'
)
clear_additional = widgets.Button(description='Zusätzliche Vektoren löschen')

# Info-Widget
vector_info = widgets.HTML(
    value='<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">'
          '<h3>Vektor Information:</h3>'
          '<p>Initialisierung...</p></div>'
)

# Hilfsfunktionen
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

def cross_product(v1, v2):
    return np.cross(v1, v2)

def angle_between(v1, v2):
    v1_n = normalize(v1)
    v2_n = normalize(v2)
    return np.arccos(np.clip(np.dot(v1_n, v2_n), -1.0, 1.0))

def project_vector(v1, v2):
    v2_n = normalize(v2)
    return np.dot(v1, v2_n) * v2_n
def update_3d_plot(*args):
    # Vektoren berechnen
    v1 = [x_slider.value, y_slider.value, z_slider.value]
    v2 = [x2_slider.value, y2_slider.value, z2_slider.value]
    v_sum = [v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2]]
    
    with fig.batch_update():
        fig.data = []  # Reset data
        
        # Koordinatenachsen
        axis_length = 10  # Vergrößert für bessere Sichtbarkeit
        fig.add_scatter3d(x=[-axis_length, axis_length], y=[0, 0], z=[0, 0],
                         mode='lines', line=dict(color='red', width=3),
                         name='X-Achse')
        fig.add_scatter3d(x=[0, 0], y=[-axis_length, axis_length], z=[0, 0],
                         mode='lines', line=dict(color='green', width=3),
                         name='Y-Achse')
        fig.add_scatter3d(x=[0, 0], y=[0, 0], z=[-axis_length, axis_length],
                         mode='lines', line=dict(color='blue', width=3),
                         name='Z-Achse')
        
        # Achsenbeschriftung wenn aktiviert
        if show_axis_numbers.value:
            for i in range(-10, 11, 2):  # Zahlen alle 2 Einheiten
                if i != 0:
                    # X-Achse
                    fig.add_scatter3d(x=[i], y=[0], z=[0], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
                    # Y-Achse
                    fig.add_scatter3d(x=[0], y=[i], z=[0], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
                    # Z-Achse
                    fig.add_scatter3d(x=[0], y=[0], z=[i], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
        
        # Erster Vektor
        fig.add_cone(x=[v1[0]], y=[v1[1]], z=[v1[2]], 
                    u=[v1[0]], v=[v1[1]], w=[v1[2]],
                    sizeref=0.3, showscale=False,  # Größerer Pfeil
                    colorscale=[[0, 'blue'], [1, 'blue']])
        fig.add_scatter3d(x=[0, v1[0]], y=[0, v1[1]], z=[0, v1[2]],
                         mode='lines', line=dict(color='blue', width=6),
                         name=f'Vektor 1 ({v1[0]:.2f}, {v1[1]:.2f}, {v1[2]:.2f})')
        
        # Zweiter Vektor wenn aktiviert
        if show_second_vector.value:
            fig.add_cone(x=[v2[0]], y=[v2[1]], z=[v2[2]], 
                        u=[v2[0]], v=[v2[1]], w=[v2[2]],
                        sizeref=0.3, showscale=False,
                        colorscale=[[0, 'red'], [1, 'red']])
            fig.add_scatter3d(x=[0, v2[0]], y=[0, v2[1]], z=[0, v2[2]],
                            mode='lines', line=dict(color='red', width=6),
                            name=f'Vektor 2 ({v2[0]:.2f}, {v2[1]:.2f}, {v2[2]:.2f})')
            
            # Summenvektor nur wenn aktiviert
            if show_sum_vector.value:
                fig.add_cone(x=[v_sum[0]], y=[v_sum[1]], z=[v_sum[2]], 
                           u=[v_sum[0]], v=[v_sum[1]], w=[v_sum[2]],
                           sizeref=0.3, showscale=False,
                           colorscale=[[0, 'green'], [1, 'green']])
                fig.add_scatter3d(x=[0, v_sum[0]], y=[0, v_sum[1]], z=[0, v_sum[2]],
                                mode='lines', line=dict(color='green', width=6),
                                name=f'Summe ({v_sum[0]:.2f}, {v_sum[1]:.2f}, {v_sum[2]:.2f})')
        
        # Layout aktualisieren
        camera = fig.layout.scene.camera
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-10, 10], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                yaxis=dict(range=[-10, 10], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                zaxis=dict(range=[-10, 10], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                aspectmode='cube',
                camera=camera
            ),
            width=800,
            height=800
        )
        
        # Vektor-Info aktualisieren
        info_html = f'''
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
            <h3>Vektor Information:</h3>
            <p><b>Vektor 1:</b> ({v1[0]:.2f}, {v1[1]:.2f}, {v1[2]:.2f})</p>
            <p><b>Längenberechnung V1:</b></p>
            <p>|v1| = √({v1[0]:.2f}² + {v1[1]:.2f}² + {v1[2]:.2f}²)</p>
            <p>|v1| = √({v1[0]**2:.2f} + {v1[1]**2:.2f} + {v1[2]**2:.2f})</p>
            <p>|v1| = {np.sqrt(v1[0]**2 + v1[1]**2 + v1[2]**2):.2f}</p>
            
            {f"<p><b>Vektor 2:</b> ({v2[0]:.2f}, {v2[1]:.2f}, {v2[2]:.2f})</p>" if show_second_vector.value else ""}
            {f"<p><b>Summenvektor:</b> ({v_sum[0]:.2f}, {v_sum[1]:.2f}, {v_sum[2]:.2f})</p>" if show_second_vector.value and show_sum_vector.value else ""}
        </div>
        '''
        vector_info.value = info_html

# Slider-Gruppen
vector1_sliders = widgets.VBox([vector1_title, x_slider, y_slider, z_slider])
vector2_sliders = widgets.VBox([vector2_title, x2_slider, y2_slider, z2_slider])
vector2_sliders.layout.visibility = 'hidden'

# Event Handler für zweiten Vektor
def on_show_second_vector_change(change):
    vector2_sliders.layout.visibility = 'visible' if change.new else 'hidden'

# Verbindungen
show_second_vector.observe(on_show_second_vector_change, names='value')
for w in [x_slider, y_slider, z_slider, x2_slider, y2_slider, z2_slider,
          show_grid, show_axis_numbers, show_second_vector, show_sum_vector]:
    w.observe(update_3d_plot, names='value')

# Layout
container = widgets.VBox([
    widgets.HBox([vector1_sliders, vector2_sliders]),
    widgets.VBox([
        widgets.HBox([show_grid]),
        widgets.HBox([show_axis_numbers]),
        widgets.HBox([show_second_vector]),
        widgets.HBox([show_sum_vector])
    ]),
    widgets.HBox([calc_cross_product, calc_angle, show_projection, clear_additional]),
    fig,
    vector_info
])

# Anzeigen
display(container)
update_3d_plot()

In [15]:
import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Figur EINMAL erstellen
fig = go.FigureWidget()

# Slider für 3D Vektor
x_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Y:')
z_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Z:')

# Slider für zweiten Vektor
x2_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='X2:')
y2_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Y2:')
z2_slider = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='Z2:')

# Checkboxen
show_second_vector = widgets.Checkbox(value=False, description='Zweiten Vektor anzeigen')
show_sum_vector = widgets.Checkbox(value=True, description='Summenvektor anzeigen')

def update_3d_plot(*args):
    v1 = [x_slider.value, y_slider.value, z_slider.value]
    v2 = [x2_slider.value, y2_slider.value, z2_slider.value]
    v_sum = [v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2]]
    
    with fig.batch_update():
        if len(fig.data) == 0:
            # Koordinatenachsen
            for i, (color, name) in enumerate([('red', 'X'), ('green', 'Y'), ('blue', 'Z')]):
                fig.add_scatter3d(
                    x=[-10, 10] if i==0 else [0,0],
                    y=[0,0] if i==0 else [-10, 10] if i==1 else [0,0],
                    z=[0,0] if i<2 else [-10, 10],
                    mode='lines',
                    line=dict(color=color, width=3),
                    name=f'{name}-Achse'
                )
                
            # Achsenbeschriftungen
            for i in range(-10, 11, 2):
                if i != 0:
                    fig.add_scatter3d(x=[i], y=[0], z=[0], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
                    fig.add_scatter3d(x=[0], y=[i], z=[0], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
                    fig.add_scatter3d(x=[0], y=[0], z=[i], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
            
            # Platzhalter für Vektoren
            for _ in range(3):
                fig.add_scatter3d(x=[0], y=[0], z=[0], mode='lines',
                                showlegend=True)
        
        # Update Vektor 1
        fig.data[-3].x = [0, v1[0]]
        fig.data[-3].y = [0, v1[1]]
        fig.data[-3].z = [0, v1[2]]
        fig.data[-3].line = dict(color='blue', width=4)
        fig.data[-3].name = f'Vektor 1'
        
        if show_second_vector.value:
            start = v1 if show_sum_vector.value else [0,0,0]
            end = [start[0]+v2[0], start[1]+v2[1], start[2]+v2[2]]
            
            fig.data[-2].x = [start[0], end[0]]
            fig.data[-2].y = [start[1], end[1]]
            fig.data[-2].z = [start[2], end[2]]
            fig.data[-2].line = dict(color='red', width=4)
            fig.data[-2].name = f'Vektor 2'
            
            if show_sum_vector.value:
                fig.data[-1].x = [0, v_sum[0]]
                fig.data[-1].y = [0, v_sum[1]]
                fig.data[-1].z = [0, v_sum[2]]
                fig.data[-1].line = dict(color='green', width=4)
                fig.data[-1].name = f'Summe'

# Layout und Display
vector1_sliders = widgets.VBox([widgets.HTML(value='<h3>Vektor 1</h3>'), 
                              x_slider, y_slider, z_slider])
vector2_sliders = widgets.VBox([widgets.HTML(value='<h3>Vektor 2</h3>'), 
                              x2_slider, y2_slider, z2_slider])
vector2_sliders.layout.visibility = 'hidden'

def on_show_second_vector_change(change):
    vector2_sliders.layout.visibility = 'visible' if change.new else 'hidden'

show_second_vector.observe(on_show_second_vector_change, names='value')
for w in [x_slider, y_slider, z_slider, x2_slider, y2_slider, z2_slider,
          show_second_vector, show_sum_vector]:
    w.observe(update_3d_plot, names='value')

container = widgets.VBox([
    widgets.HBox([vector1_sliders, vector2_sliders]),
    widgets.HBox([show_second_vector, show_sum_vector]),
    fig
])

# Initiales Layout setzen
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[-10, 10], showgrid=True, showbackground=False,
                  gridcolor='lightgray'),
        yaxis=dict(range=[-10, 10], showgrid=True, showbackground=False,
                  gridcolor='lightgray'),
        zaxis=dict(range=[-10, 10], showgrid=True, showbackground=False,
                  gridcolor='lightgray'),
        aspectmode='cube'
    ),
    width=800,
    height=800
)

display(container)
update_3d_plot()